# Collection of Data for Snowpack Layer Prediction and Analysis
### Jaymin West

### Spring, 2023

#### The purpose of this project is to create a model that predicts the current snow conditions in areas based on the weather that that area has seen and predict the avalanche risk of that area based on this. By “snow conditions” I mean the layers that exist within the snow pack such as hard, frozen layers or soft, dry layers. Understanding the layers within a snowpack is essential to predicting avalanche risk of an area. Currently, avalanche risk assessment and snow pack analysis are done entirely by professional avalanche forecasters who go into the field, dig snow pits, analayze the snowpack, and create a risk assessment based on this information. The goal with this project is not to replace these highly educated individuals but instead to attempt to create a tool that may help those interested in snow packs better understand the conditions they may face.

In [2]:
import importlib, os, utils, urllib
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Stations, Daily, Point, Hourly
import sqlalchemy
import pandas as pd
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
importlib.reload(utils)
all_zones_df = utils.clean_raw_webscraper_data('output_data/All_Zones_2021-22_Season_reports_data.csv')


bottom_line_text
problem_type_text
forecast_discussion_text
combined_text


In [7]:
importlib.reload(utils)
all_zones_df = all_zones_df.head(1)
all_zones_df = utils.add_weather_to_reports(all_zones_df)
all_zones_df.head()
utils.dataframe_to_postgres(all_zones_df)

,date,zone,overall_risk,above_treeline_risk,near_treeline_risk,below_treeline_risk,bottom_line_text,problem_type_text,forecast_discussion_text,combined_text,tavg,tmin,tmax,prcp,wdir,pres,tsun
0,2022-04-23,east slopes north,3.0,3.0,3.0,2.0,avalanche danger minimal morning quickly spike...,expect warmest temperature almost week enough ...,cornice fall snow blanket cornice rock face ne...,avalanche danger minimal morning quickly spike...,7.6,-0.9,15.9,0.0,225.3,1024.0,0.0


### Creating an LSTM to predict the current avalanche danger

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool

p = figure(title="Avalanche Risk 22-23", x_axis_label='Date', y_axis_label='Value')

p.line(x='time', y='temp', legend_label="Temperature", color='red', source=weather_and_risk_df)
# p.line(x='time', y='dwpt', legend_label="Dew Point", source=weather_and_risk_df)
# p.line(x='time', y='rhum', legend_label="Relative Humidity", source=weather_and_risk_df)
# p.line(x='time', y='prcp', legend_label="Precipitation", source=weather_and_risk_df)
# p.line(x='time', y='wdir', legend_label="Wind Direction", source=weather_and_risk_df)
p.line(x='time', y='wspd', legend_label="Wind Speed", source=weather_and_risk_df)
# p.line(x='time', y='pres', legend_label="Pressure", source=weather_and_risk_df)
# p.line(x='time', y='coco', legend_label="Cloud Cover", source=weather_and_risk_df)

hover = HoverTool(tooltips=[("Avalanche Risk", "@risk")], mode='vline')
p.add_tools(hover)

output_file("output_data/avalanche_risk_22-23.html")

show(p)

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/jaymin/snap/code/common/.cache/gio-modules/libgiolibproxy.so
libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)


Opening in existing browser session.


In [ ]:
weather_and_risk_df['time'] = pd.to_datetime(weather_and_risk_df['time'])
weather_and_risk_df['time'] = weather_and_risk_df['time'].apply(lambda x: x.timestamp())

X = weather_and_risk_df.iloc[:, :-1].values
y = weather_and_risk_df.iloc[:, -1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

n_samples, n_time_steps = X.shape
n_features = 1
X = X.reshape((n_samples, n_time_steps, n_features))

train_sie = int(0.8 * len(X))
X_train, X_test = X[:train_sie], X[train_sie:]
y_train, y_test = y[:train_sie], y[train_sie:]

model = Sequential()
# model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_time_steps, n_features)))
model.add(Bidirectional(LSTM(25, activation='relu'), input_shape=(n_time_steps, n_features)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=16, batch_size=16, verbose=1)

mse = model.evaluate(X_test, y_test, verbose=0)
print('Test MSE: %.3f' % mse)

Epoch 1/16
8/8 [==============================] - 5s 11ms/step - loss: 2.1643
Epoch 2/16
8/8 [==============================] - 0s 12ms/step - loss: 1.9405
Epoch 3/16
8/8 [==============================] - 0s 12ms/step - loss: 1.7272
Epoch 4/16
8/8 [==============================] - 0s 11ms/step - loss: 1.4953
Epoch 5/16
8/8 [==============================] - 0s 11ms/step - loss: 1.2299
Epoch 6/16
8/8 [==============================] - 0s 11ms/step - loss: 0.9206
Epoch 7/16
8/8 [==============================] - 0s 11ms/step - loss: 0.6316
Epoch 8/16
8/8 [==============================] - 0s 16ms/step - loss: 0.6196
Epoch 9/16
8/8 [==============================] - 0s 14ms/step - loss: 0.6109
Epoch 10/16
8/8 [==============================] - 0s 15ms/step - loss: 0.5933
Epoch 11/16
8/8 [==============================] - 0s 13ms/step - loss: 0.6004
Epoch 12/16
8/8 [==============================] - 0s 10ms/step - loss: 0.6023
Epoch 13/16
8/8 [==============================] - 0s 12ms/st

In [ ]:
weather_and_risk_df

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,risk
0,2022-11-04,2.6,1.6,92.9,78.6,319.2,6.9,1008.0,16.0,-1
1,2022-11-16,-4.7,-7.0,84.8,0.0,74.7,8.0,1037.2,5.0,-1
2,2022-11-18,-6.5,-10.6,73.2,0.0,88.5,9.0,1033.9,1.0,-1
3,2022-11-21,-4.8,-8.7,74.2,0.0,66.4,6.4,1026.0,3.0,-1
4,2022-11-23,1.1,-0.2,91.6,4.2,281.6,8.4,1022.1,16.0,-1
...,...,...,...,...,...,...,...,...,...,...
145,2023-04-12,-2.0,-4.4,83.8,0.5,322.2,4.0,1015.1,21.0,1
146,2023-04-13,-1.1,-4.6,78.3,0.0,277.7,7.5,1015.2,5.0,1
147,2023-04-14,0.6,-3.7,75.4,0.0,279.2,5.6,1013.3,5.0,0
148,2023-04-15,2.2,-3.0,69.0,0.3,51.9,4.4,1018.2,7.0,1


In [ ]:
# Getting data from the database:
import psycopg2
conn = psycopg2.connect("dbname='snowpackprediction' user='jaymin' host='localhost' password='password'")
conn.autocommit = True
cursor = conn.cursor()


weather_df = pd.DataFrame(cursor.execute("SELECT * FROM weather_data"))
weather_df.head()

""


In [ ]:
X = weather_and_risk_df[['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres', 'coco']] # Features
y = weather_and_risk_df['risk'] # Labels

# Making a decision tree with Sklearn:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X[:-10], y[:-10])
# fig = plt.figure(figsize=(25,20))
# fig = tree.plot_tree(clf)
clf.predict(X[-10:])
print("Decision Tree Score: ", clf.score(X[-10:], y[-10:]))
# conf_matrix = confusion_matrix(clf.predict(X), y)

# fig, ax = plt.subplots(figsize=(3, 3))
# ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
# for i in range(conf_matrix.shape[0]):
#     for j in range(conf_matrix.shape[1]):
#         ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()


Decision Tree Score:  0.4


In [ ]:
# Scraping the avalanche data from the NWAC website:

# Xpath: //*[@id="afp-forecast-widget"]/div/div/div[1]/div[2]/div[1]/table/tbody/tr[1]/td[1]/div/a

# Using Selenium to get the avalanche forecast data:
date_risks = []

browser = webdriver.Chrome()
url = 'https://nwac.us/avalanche-forecast/#/archive'
browser.get(url)
# Finding by XPATH:
select_element = Select(browser.find_element(By.XPATH,'//*[@id="afp-forecast-widget"]/div/div/div[1]/div[1]/div/div[1]/div[2]/div[2]/select'))
# Selecting Snoqualmie Pass from dropdown menu:
select_element.select_by_visible_text("Snoqualmie Pass")

response = browser.page_source

soup = bs.BeautifulSoup(response, 'html.parser')

prediction_table = soup.find_all('tr', {'class': 'VueTables__row'})
for row in prediction_table:
    report_text = row.find_all('a')[0].text
    # report = row.find_all('a')[0]['href']
    report = Select(browser.find_element(By.LINK_TEXT(report_text)))
    print(report)
    date = row.find_all('td')[0].text

    org_date = datetime.strptime(date, '%b %d, %Y')
    new_date = datetime.strftime(org_date, '%Y-%m-%d')
    new_date = datetime.strptime(new_date, '%Y-%m-%d')
    date_risks.append([new_date, row.find_all('td')[5].text])

# Going to the next page:
select_element = browser.find_element(By.XPATH,'//*[@id="afp-forecast-widget"]/div/div/div[1]/div[2]/div[2]/nav/ul/li[4]/a')
select_element.click()

response = browser.page_source

soup = bs.BeautifulSoup(response, 'html.parser')

prediction_table = soup.find_all('tr', {'class': 'VueTables__row'})
for row in prediction_table:
    date = row.find_all('td')[0].text
    org_date = datetime.strptime(date, '%b %d, %Y')
    new_date = datetime.strftime(org_date, '%Y-%m-%d')
    new_date = datetime.strptime(new_date, '%Y-%m-%d')
    date_risks.append([new_date, row.find_all('td')[5].text])

browser.quit()

date_risks = pd.DataFrame(date_risks, columns=['time', 'risk'])

TypeError: 'str' object is not callable

# Snowpilot Data Collection

In [ ]:
importlib.reload(utils)

sp_data = []
for filename in os.listdir("input_data/snowpilot_data"): 
    filename = "input_data/snowpilot_data/" + filename
    timestamp = int(utils.snowpilot_xml_to_dict(filename)['@timestamp'])
    timestamp /= 1000 # Converting from milliseconds

    date = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')

    layers = utils.snowpilot_xml_to_dict(filename)['Layer']

    layers_dict = {}

    for layer in layers:
        layers_dict[layer['@layerNumber']] = [layer['@startDepth'], layer['@endDepth'], layer['@hardness1']]

    chart_df = pd.DataFrame.from_dict(layers_dict, orient='index', columns=['startDepth', 'endDepth', 'hardness1'])
    sp_data.append((date, chart_df))
sp_data

ExpatError: syntax error: line 1, column 0

## Ideas:

- Really only need to predict the hardness of the snow layers. Graph can be made from everything else
- Snow depth can be retrieved from the Snowpilot Data
- Take basically all of the attributes possible from the weather data, use decision tree to find the most influential factors in determinding the layer hardness
    - Will need to look at (some) historical data for the best results here
- End results does not have to be the same format of the Snowpilot charts
    - Could have the layers be color coded